In [1]:
from pyomo.environ import *
import mpisppy.utils.sputils as sputils
import matplotlib.pyplot as plt
from matplotlib import rc
import sys
sys.path.append('../../../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set
from energiapy.utils.scale_utils import scale_list, scale_tuple

[    0.00] Initializing mpi-sppy


In [2]:
_time_intervals = 30  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 12  # Number of execution scenarios                     (chi)

M = 1e3  # Big M 

In [3]:
def build_model(cap_factor):
    
    # Define temporal scales
    scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=125, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system')
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    # com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    prod_max = {0:30, 1: 60, 2: 90, 3: 120, 4: 150}
    prod_min = {0:0, 1: 30, 2: 60, 3: 90, 4: 120}
    rate_max = {0:1.25, 1: 1, 2: 0.75, 3: 0.5, 4: 0.25}
    mode_ramp = {(0,1): 5, (1,2): 5}
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 150
    
    com1_procure = Process(name='procure com1', prod_max=125, conversion={com1_pur: -1, com1_in: 1}, capex=0.01, vopex=0.01, prod_min=0.01, label='Procure com1', varying=[VaryingProcess.DETERMINISTIC_CAPACITY])
    com1_sell = Process(name='sell com1', prod_max=com1_process_capacity, conversion={com1_out: -1, com1_sold: 1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Sell com1')
    com1_opt_procure = Process(name='procure optional com1', prod_max=125, conversion={com1_pur: -1, com1_in:1}, capex=10, vopex=0.1, prod_min=0.01, label='Procure optional com1')
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 1')
    # com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 2')
    
    # com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Process the commodity through the location')
    com1_process = Process(name='com1_process', prod_max=prod_max, conversion={0:{com1_in: -1, com1_out: 1}, 1:{com1_in: -1, com1_out: 1}, 2:{com1_in: -1, com1_out: 1}, 3:{com1_in: -1, com1_out: 1}, 4:{com1_in: -1, com1_out: 1}},  capex=0.01, vopex=0.01, prod_min=prod_min, rate_max=rate_max, varying=[VaryingProcess.DETERMINISTIC_CAPACITY], label='Process the commodity through the location')
    
    # com1_store10 = Process(name='com1_store10', prod_max=150, capex=100, vopex=1, store_min=0.01, store_max= 20, prod_min=150, label="Storage capacity of 10 units", storage=com1_in, storage_cost=0.02)
    com1_store20 = Process(name='com1_store20', prod_max=150, capex=200, vopex=2, store_min=0.01, store_max= 40, prod_min=150, label="Storage capacity of 20 units", storage=com1_in, storage_cost=0.02)
    # com1_store50 = Process(name='com1_store50', prod_max=150, capex=500, vopex=5, store_min=0.01, store_max= 100, prod_min=150, label="Storage capacity of 50 units", storage=com1_in, storage_cost=0.02)
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 1')
    # com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 2')
    
    
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_process, com1_store20, com1_loc1_send, com1_sell, com1_opt_procure}, label="Location 1", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, capacity_factor={com1_procure: cap_factor[['com1_procure']]})
    
    # loc2 = Location(name='loc2', processes={com1_receive_loc1, com1_process, com1_store20, com1_loc2_send, com1_sell, com1_opt_procure}, label="Location 2", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    
    # ======================================================================================================================
    # Declare transport/trucks
    # ======================================================================================================================
    
    # truck_cap12 = 70
    
    # truck12 = Transport(name='truck12', resources={com1_loc1_out}, trans_max=truck_cap12, label='Truck from location 1 to 2', capex=0.5, vopex=0.05, trans_min=truck_cap12)
    # truck21 = Transport(name='truck21', resources={com1_loc2_out}, trans_max=truck_cap12, label='Truck from location 2 to 1', capex=0.0001, vopex=0.05, trans_min=truck_cap12)
    
    
    
    # ======================================================================================================================
    # Declare network
    # ======================================================================================================================

    # transport_matrix = [
    #     [[], [truck12]],  # source: location 1
    #     [[truck21], []]  # source: location 2
    # ]
    # 
    # distance_matrix = [
    #     [0, 55],
    #     [55, 0]
    # ]
    # 
    locset = [loc1]
    # 
    # sources = locset
    # sinks = locset
    # 
    # network = Network(name='Network', scales= scales, source_locations=sources, sink_locations=sinks, transport_matrix=transport_matrix, distance_matrix=distance_matrix)



    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = 100
    demand_penalty = 50
    
    demand_dict = {i: {com1_sold: daily_demand} if i == loc1 else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty} if i == loc1 else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name='scenario_baseline', network= loc1, scales=scales, scheduling_scale_level=2, network_scale_level=0, purchase_scale_level=2, availability_scale_level=1, demand_scale_level=2, capacity_scale_level=1, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Scenario with perfect information')
    
    # ======================================================================================================================
    # Declare problem
    # ======================================================================================================================
    
    problem_mincost = formulate(scenario=scenario,
                            constraints={Constraints.COST, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK},
                            demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)
    
    
    scale_iter = scale_tuple(instance=problem_mincost, scale_levels=scenario.network_scale_level+1)
    capex_process= sum(problem_mincost.Capex_network[scale_] for scale_ in scale_iter)
    
    problem_mincost.first_stage_cost  = capex_process
    
    return scenario, problem_mincost

In [4]:
#Expected Cost under Perfect Information
all_scenario_names = ["good", "average", "bad"]
ns_dict = {'good': 12, "average": 9, "bad":6}
scenario_probabilities = {'good': 0.25, 'average': 0.5, 'bad':0.25} 

exCost_PI = 0

for scenario_name in all_scenario_names:
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen, model = build_model(cap_factor=cap_factor)
    for loc in model.locations:
        model.X_P[loc,"procure com1",scen.network_scale_level].fixed= True
        model.X_P[loc,"procure com1",scen.network_scale_level].value= 1
    solver = SolverFactory("gurobi")
    solver.solve(model)
    
    exCost_PI += value(model.objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint production mode
ERROR: Rule failed when generating expression for Constraint
constraint_nameplate_production_varying_capacity with index ('loc1',
'com1_process', 0, 0, 0): KeyError: 'com1_process'
ERROR: Constructing component
'constraint_nameplate_production_varying_capacity' from data=None failed:
        KeyError: 'com1_process'


KeyError: 'com1_process'

In [ ]:
exCost_PI

In [ ]:
def scenario_creator(scenario_name):
    if scenario_name=='good':
        ns = 12
    elif scenario_name=='average':
        ns = 9
    elif scenario_name=='bad':
        ns = 6
    else:
        raise ValueError("Unrecognized scenario name")
    
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns + [0]*(_exec_scenarios-ns)})
    scen, model = build_model(cap_factor=cap_factor)
    sputils.attach_root_node(model, model.first_stage_cost, [model.X_P])
    model._mpisppy_probability = scenario_probabilities[scenario_name]
    return model

In [ ]:
from mpisppy.opt.ef import ExtensiveForm

options = {"solver": "gurobi"}
all_scenario_names = ["good", "average", "bad"]
ef_UI = ExtensiveForm(options, all_scenario_names, scenario_creator)
results = ef_UI.solve_extensive_form()

In [ ]:
exCost_UI = ef_UI.get_objective_value()
print(f"{exCost_UI:.3f}")

In [ ]:
EVPI = exCost_UI - exCost_PI
EVPI

In [ ]:
soln = ef_UI.get_root_solution()
for (var_name, var_val) in soln.items():
    print(var_name, var_val)

In [ ]:
# from mpisppy.opt.lshaped import LShapedMethod
# 
# all_scenario_names = ["good", "average", "bad"]
# bounds = {name: -432000 for name in all_scenario_names}
# options = {
#     "root_solver": "gurobi",
#     "sp_solver": "gurobi",
#     "sp_solver_options" : {"threads" : 1},
#     "valid_eta_lb": bounds,
#     "max_iter": 10,
# }
# 
# ls = LShapedMethod(options, all_scenario_names, scenario_creator)
# result = ls.lshaped_algorithm()
# 
# variables = ls.gather_var_values_to_rank0()
# for ((scen_name, var_name), var_value) in variables.items():
#     print(scen_name, var_name, var_value)

In [ ]:
n = 9
cap_factor = pandas.DataFrame(data={'com1_procure': [1]*n + [0]*(_exec_scenarios-n)})
scen, model = build_model(cap_factor=cap_factor)

for loc in model.locations:
        model.X_P[loc,"procure com1",scen.network_scale_level].fixed= True
        model.X_P[loc,"procure com1",scen.network_scale_level].value= 1

solver = SolverFactory("gurobi")
solver.solve(model)

In [ ]:
print(f"{value(model.objective_cost_w_demand_penalty):.3f}")

In [ ]:
model.X_P.pprint()

In [ ]:
#Value of Stochastic Solution
all_scenario_names = ["good", "average", "bad"]
ns_dict = {'good': 12, "average": 9, "bad": 6}
exCost_FD = 0

for scenario_name in all_scenario_names:
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen, model = build_model(cap_factor=cap_factor)
    for process in scen.location_process_dict['loc1']:
        model.X_P['loc1', process, scen.network_scale_level].fixed= True

    model.X_P['loc1','com1_loc1_send',0].value = 0.0
    model.X_P['loc1','com1_process',0].value = 1.0
    model.X_P['loc1','com1_store20',0].value = 0.0
    model.X_P['loc1','com1_store20_discharge',0].value = 1.0
    model.X_P['loc1','procure com1',0].value = 1.0
    model.X_P['loc1','procure optional com1',0].value = 1.0
    model.X_P['loc1','sell com1',0].value = 1.0
    
    solver = SolverFactory("gurobi")
    solver.solve(model)
    
    exCost_FD += value(model.objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

In [ ]:
VSS = exCost_FD - exCost_UI
VSS